# 03_model_training.ipynb - CNN Vehicle Document Classifier

## Objetivo
Entrenar un modelo CNN para clasificación de documentos vehiculares (licence/odometer/document) con >80% accuracy.

## Estado del Proyecto
- ✅ Dataset balanceado: 729 muestras, 3 clases
- ✅ Preprocesamiento completo con OpenCV
- ✅ Splits train/val/test listos
- 🎯 Objetivo: Modelo CNN con TensorFlow/Keras

## 1. Setup and Imports

In [1]:
# 1. Setup and Imports (Optimizado para CPU)

# Suprimir warnings de TensorFlow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suprime warnings de TF
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Fuerza uso de CPU

# System imports
import json
import random
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2  # Más ligero para CPU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

# Model evaluation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

# MLOps and utilities
import mlflow
import mlflow.tensorflow
from tqdm import tqdm

# Configurar TensorFlow para CPU
tf.config.set_visible_devices([], 'GPU')

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print(f"TensorFlow version: {tf.__version__}")
print(f"Running on: CPU")
print(f"Number of CPU threads: {tf.config.threading.get_intra_op_parallelism_threads()}")

TensorFlow version: 2.13.0
Running on: CPU
Number of CPU threads: 0


2025-07-05 14:58:14.870889: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
# 2. Configuration and Constants (Optimizado para CPU)

# Project paths
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data" / "processed" / "car_plates"
ANNOTATIONS_DIR = DATA_DIR / "annotations"
IMAGES_DIR = DATA_DIR / "images_rois"
MODELS_DIR = BASE_DIR / "models"
LOGS_DIR = BASE_DIR / "logs"

# Create directories if they don't exist
MODELS_DIR.mkdir(exist_ok=True, parents=True)
LOGS_DIR.mkdir(exist_ok=True, parents=True)

# Model configuration optimizada para CPU
CONFIG = {
    # Data
    'image_size': (160, 160),  # Tamaño reducido para CPU
    'num_classes': 3,
    'class_names': ['document', 'licence', 'odometer'],
    
    # Training - Ajustado para CPU
    'batch_size': 16,  # Batch más pequeño para CPU
    'epochs': 30,  # Menos epochs inicialmente
    'learning_rate': 1e-3,
    'early_stopping_patience': 7,
    'reduce_lr_patience': 3,
    
    # Augmentation (más ligero)
    'rotation_range': 10,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'zoom_range': 0.05,
    'horizontal_flip': False,
    
    # Model architecture
    'dropout_rate': 0.4,  # Más dropout para dataset pequeño
    'l2_regularization': 1e-4,
    
    # MLOps
    'experiment_name': 'vehicle_doc_classifier_cpu',
    'run_name': f"cnn_cpu_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
}

print("Configuration loaded successfully!")
print(f"Image size: {CONFIG['image_size']}")
print(f"Batch size: {CONFIG['batch_size']} (optimized for CPU)")


Configuration loaded successfully!
Image size: (160, 160)
Batch size: 16 (optimized for CPU)


In [3]:
# 3. Arquitectura de modelo ligera para CPU

def create_lightweight_cnn_model(input_shape=(160, 160, 3), num_classes=3):
    """
    Modelo CNN ligero optimizado para CPU y dataset pequeño
    Usa MobileNetV2 que es más eficiente en CPU
    """
    
    # Base model - MobileNetV2 es más eficiente en CPU
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet',
        alpha=0.75  # Versión más ligera
    )
    
    # Congelar solo las primeras capas (fine-tuning más agresivo para dataset pequeño)
    for layer in base_model.layers[:-30]:
        layer.trainable = False
    
    # Build the model
    inputs = keras.Input(shape=input_shape)
    
    # Preprocessing
    x = keras.applications.mobilenet_v2.preprocess_input(inputs)
    
    # Base model
    x = base_model(x, training=False)
    
    # Custom top layers (más simples para CPU)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu',
                     kernel_regularizer=keras.regularizers.l2(CONFIG['l2_regularization']))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(CONFIG['dropout_rate'])(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs)
    
    return model, base_model

In [5]:
# Optimizaciones adicionales para CPU
def optimize_tensorflow_for_cpu():
    """Configuraciones para mejorar rendimiento en CPU"""
    
    # Configurar paralelismo
    tf.config.threading.set_intra_op_parallelism_threads(4)  # Usa todos los cores
    tf.config.threading.set_inter_op_parallelism_threads(4)  # Usa todos los cores
    
    # Habilitar XLA para optimización
    tf.config.optimizer.set_jit(True)
    
    print("✓ TensorFlow optimizado para CPU")
    print(f"✓ Cores disponibles: {os.cpu_count()}")

optimize_tensorflow_for_cpu()

✓ TensorFlow optimizado para CPU
✓ Cores disponibles: 4
